In [ ]:
pip install tensorflow tensorflow-addons

Used Dummy Unet Model for Segmentation

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the pre-trained U-Net model
segmentation_model = load_model('unet_model.h5')

def segment_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256, 256))  # Resize to match the model's input size
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    segmented_img = segmentation_model.predict(img)
    segmented_img = np.squeeze(segmented_img, axis=0)  # Remove batch dimension
    segmented_img = (segmented_img > 0.5).astype(np.uint8)  # Threshold the output
    return segmented_img


Example Usage: It serves as a quick example or test case to show how to apply the function segment_and_extract to an individual image. Functionality Test: It verifies that the segment_and_extract function works correctly by running it on a sample image and obtaining the extracted features. Feature Extraction: It extracts the features from the provided image using the segmentation model (a placeholder in the current code) and the feature extraction model (ResNet101).

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import Adam


# Load the pre-trained ResNet101 model without the top fully connected layers
base_model = ResNet101(weights='imagenet', include_top=False, pooling='avg')

def preprocess_image(image):
    img = cv2.resize(image, (224, 224))  # Resize to match ResNet input size
    if img.shape[-1] == 1:  # Convert grayscale to RGB if needed
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Preprocess for ResNet
    return img

def extract_features(image, model):
    preprocessed_image = preprocess_image(image)
    features = model.predict(preprocessed_image)
    return features

def segment_image(image_path):
    # Dummy function for image segmentation, replace with actual segmentation code
    return cv2.imread(image_path)

def segment_and_extract(image_path):
    segmented_img = segment_image(image_path)
    features = extract_features(segmented_img, base_model)
    return features

# Example usage
image_path = 'C:/Users/User/Desktop/back up/my doc/RESEARCH/CINNAMON CODES/test.jpeg'
features = segment_and_extract(image_path)

dataset_dir = r'C:\Users\User\Desktop\back up\my doc\RESEARCH\CINNAMON CODES\Dataset'
classes = [class_name for class_name in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, class_name))]

X = []
y = []

# Initialize Label Binarizer for one-hot encoding the labels
lb = LabelBinarizer()
lb.fit(classes)

for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.isdir(class_dir):
        continue
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        segmented_img = segment_image(image_path)
        features = extract_features(segmented_img, base_model)
        X.append(features[0])  # Extracted features are 2D, take the first dimension
        y.append(class_name)

# Convert lists to numpy arrays
X = np.array(X)
y = lb.transform(y)

# Optionally, save the extracted features and labels for future use
np.save('X_features.npy', X)
np.save('y_labels.npy', y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of classes
num_classes = len(classes)



# Define your classifier
model = Sequential([
    Dense(256, activation='relu', input_shape=(2048,)),  # Assuming ResNet101 output features of size 2048
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')  # num_classes should be the number of classes you have
])
# Define your learning rate
learning_rate = 0.0001  # Adjust this value as needed

# Instantiate the Adam optimizer with the desired learning rate
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))


In [ ]:
# Save the classifier model
model.save('classifier_model.h5')



In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy}")


In [ ]:
# Function to predict the class of a new image
def predict_image_class(image_path, segmentation_model, feature_extraction_model, classifier_model):
    segmented_img = segment_image(image_path)
    features = extract_features(segmented_img, feature_extraction_model)
    prediction = classifier_model.predict(features)
    predicted_class = lb.inverse_transform(prediction)[0]
    return predicted_class

# Example usage for prediction
new_image_path = 'C:/Users/User/Desktop/back up/my doc/RESEARCH/CINNAMON CODES/test_image.jpg'
classifier_model = load_model('classifier_model.h5')
predicted_class = predict_image_class(new_image_path, segmentation_model, base_model, classifier_model)
print(f"The predicted class is: {predicted_class}")

In [ ]:
# Function to predict the class of a new image
def predict_image_class(image_path, segmentation_model, feature_extraction_model, classifier_model):
    segmented_img = segment_image(image_path)
    features = extract_features(segmented_img, feature_extraction_model)
    prediction = classifier_model.predict(features)
    predicted_class = lb.inverse_transform(prediction)[0]
    return predicted_class

# Example usage for prediction
new_image_path = 'C:/Users/User/Desktop/back up/my doc/RESEARCH/CINNAMON CODES/uploads/WhatsApp Image 2024-06-25 at 17.37.43.jpeg'
classifier_model = load_model('classifier_model.h5')
predicted_class = predict_image_class(new_image_path, segmentation_model, base_model, classifier_model)
print(f"The predicted class is: {predicted_class}")